In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from pathlib import Path
import re
from nltk.corpus import stopwords
import nltk
import grpc
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g
from functools import lru_cache

# Gerekli NLTK bileşenleri
nltk.download('stopwords')

# Zemberek GRPC bağlantısı
channel = grpc.insecure_channel('localhost:6789')
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Melek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy, scipy, gensim
print(numpy.__version__, scipy.__version__, gensim.__version__)

1.26.4 1.13.1 4.3.3


In [4]:
# Önbellekli kök bulma fonksiyonu
@lru_cache(maxsize=10000)
def get_stem(word):
    try:
        response = morphology_stub.AnalyzeWord(z_morphology.WordAnalysisRequest(input=word))
        if response.analyses:
            return response.analyses[0].lemmas[0]  # İlk lemma
        return word
    except grpc.RpcError as e:
        print(f"Zemberek hatası: {e}")
        return word


# Türkçe için stopword'ler
stop_words = set(stopwords.words("turkish"))

In [5]:
# Veri setini yükle
data_path = Path(r"C:\Users\Melek\yapayZeka\FilmTemaAnaliziProje\İşlenmişVeriler3\tum_filmler_etiketli_stemmed.csv")
df = pd.read_csv(data_path, encoding='utf-8-sig')

# Boş olmayan cümleleri filtrele
df = df[df['Processed_Sentence'].notna() & (df['Processed_Sentence'] != '')]

print("Başlangıç veri seti boyutu:", len(df))
print("Sınıf dağılımı:\n", df['Etiket'].value_counts())



Başlangıç veri seti boyutu: 74765
Sınıf dağılımı:
 Etiket
komedi         6384
suç            6251
spor           5728
romantik       5482
fantastik      5166
müzik          5088
animasyon      4910
gerilim        4906
dram           4680
tarih          4357
bilim kurgu    4277
aksiyon        4004
polisiye       3819
korku          3682
distoptik      3439
savaş          2592
Name: count, dtype: int64


In [6]:
# Metin ön işleme fonksiyonu (Zemberek entegrasyonlu)
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\sğüşıöçĞÜŞİÖÇ]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    words = text.split()
    words = [get_stem(word) for word in words if word not in stop_words]

    return " ".join(words)

In [7]:
df['Processed_Text'] = df['Processed_Sentence'].apply(preprocess_text)

In [8]:
# İyileştirilmiş Overlap'li chunklama + akıllı etiketleme
chunk_size = 25  # Biraz artırıldı, daha bütünsel bilgi için
stride = 12  # Daha fazla örtüşme, veri çeşitliliği için
chunked_texts = []

for film, group in df.groupby('Film_Name'):
    sentences = group['Processed_Text'].tolist()
    labels = group['Etiket'].tolist()

    for i in range(0, len(sentences) - chunk_size + 1, stride):
        chunk = sentences[i:i + chunk_size]
        chunk_labels = labels[i:i + chunk_size]
        
        # Etiket dağılımını hesapla
        label_counts = pd.Series(chunk_labels).value_counts()
        
        # Chunk'ın en az %70'i aynı etikete sahipse o etiketi kullan
        majority_label = label_counts.index[0]
        majority_pct = label_counts.iloc[0] / len(chunk_labels)
        
        if majority_pct >= 0.7:
            chunk_label = majority_label
        else:
            # Aksi takdirde mod kullan
            chunk_label = pd.Series(chunk_labels).mode().iloc[0]

        chunked_texts.append({
            'Film_Name': film,
            'Etiket': chunk_label,
            'Text': ' '.join(chunk)
        })

chunked_df = pd.DataFrame(chunked_texts)

In [9]:
# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    chunked_df['Text'],
    chunked_df['Etiket'],
    test_size=0.2,
    random_state=42,
    stratify=chunked_df['Etiket']
)

print(f"Oluşturulan chunk sayısı: {len(chunked_df)}")
print(f"Eğitim seti boyutu: {len(X_train)}")
print(f"Test seti boyutu: {len(X_test)}")

Oluşturulan chunk sayısı: 6158
Eğitim seti boyutu: 4926
Test seti boyutu: 1232


In [10]:
# İnce ayarlı optimum pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        min_df=6,         # Biraz daha spesifik olması için artırıldı
        max_df=0.65,      # Orta seviye ayar
        ngram_range=(1, 3),  # 1-3 gram - daha iyi dil örüntüleri
        max_features=5000,  # Biraz artırıldı 
        sublinear_tf=True   # Logaritmik TF skalama
    )),
    ('clf', LogisticRegression(
        class_weight='balanced',
        max_iter=10000,
        solver='saga',     # Büyük veri setleri için en iyi
        penalty='elasticnet', # Hem L1 hem L2 regularizasyonu  
        l1_ratio=0.15,     # Daha hafif L1 - daha az özellik elemesi
        C=0.25            # Biraz daha yüksek - gerekli örüntüleri öğrenmek için
    ))
])

In [11]:
from sklearn.model_selection import GridSearchCV

# İnce ayarlı parametre gridleri
param_grid = {
    'tfidf__max_features': [4500, 5000, 5500],
    'clf__C': [0.2, 0.25, 0.3],
    'clf__l1_ratio': [0.1, 0.15, 0.2]
}

In [12]:
from sklearn.model_selection import StratifiedKFold

# K-fold cross-validation ile model eğitimi
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, verbose=1, n_jobs=-1, 
                          scoring='f1_weighted')

In [13]:
print("Model eğitiliyor...")
grid_search.fit(X_train, y_train)

# En iyi parametreleri göster
print("\nEn iyi hiperparametreler:")
print(grid_search.best_params_)
print(f"En iyi cross-validation skoru: {grid_search.best_score_:.4f}")

# En iyi modeli al
best_model = grid_search.best_estimator_


Model eğitiliyor...
Fitting 5 folds for each of 27 candidates, totalling 135 fits

En iyi hiperparametreler:
{'clf__C': 0.3, 'clf__l1_ratio': 0.1, 'tfidf__max_features': 5500}
En iyi cross-validation skoru: 0.9024


In [14]:
from sklearn.metrics import confusion_matrix

# Değerlendirme
# Test verisi üzerinde tahmin
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Eğitim verisi üzerinde tahmin (overfitting kontrolü için)
y_pred_train = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)

print("\n" + "="*50)
print("Eğitim Verisi Performansı:")
print(f"Eğitim Doğruluğu (Train Accuracy): {train_accuracy:.4f}")
print("\nEğitim Verisi Sınıflandırma Raporu:")
print(classification_report(y_train, y_pred_train))

print("\n" + "="*50)
print("Test Verisi Performansı:")
print(f"Test Doğruluğu (Test Accuracy): {test_accuracy:.4f}")
print("\nTest Verisi Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_test))
print("="*50 + "\n")

# Overfitting analizi
train_test_diff = train_accuracy - test_accuracy
print(f"ℹ️ Bilgi: Eğitim-Test farkı: {train_test_diff:.4f}")

if train_accuracy > 0.95 and train_test_diff > 0.2:
    print("⚠️ UYARI: Model aşırı öğrenme (overfitting) yapmış görünüyor!")
elif train_test_diff < 0.08:
    print("✅ Model iyi genelleme yapıyor gibi görünüyor")
else:
    print("ℹ️ Modelde hafif overfitting olabilir")

# Sınıf bazlı performans analizi
test_report = classification_report(y_test, y_pred_test, output_dict=True)
class_f1 = {cls: scores['f1-score'] for cls, scores in test_report.items() 
          if cls not in ['accuracy', 'macro avg', 'weighted avg']}

# En kötü 3 sınıfı bul
worst_classes = sorted(class_f1.items(), key=lambda x: x[1])[:3]
print("\nEn Düşük Performans Gösteren Sınıflar:")
for cls, score in worst_classes:
    print(f"{cls}: F1-score = {score:.4f}")

# En iyi 3 sınıfı bul
best_classes = sorted(class_f1.items(), key=lambda x: x[1], reverse=True)[:3]
print("\nEn Yüksek Performans Gösteren Sınıflar:")
for cls, score in best_classes:
    print(f"{cls}: F1-score = {score:.4f}")

# Karışıklık matrisi analizi
print("\nKarışıklık matrisini hesaplama ve görselleştirme...")

def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Yalnızca en çok karıştırılan sınıfları göster
    misclassification = []
    for i in range(len(classes)):
        for j in range(len(classes)):
            if i != j and cm[i, j] > 0:
                misclassification.append((classes[i], classes[j], cm[i, j], cm_norm[i, j]))
    
    # En çok karıştırılan 10 sınıf çifti
    top_conf = sorted(misclassification, key=lambda x: x[2], reverse=True)[:10]
    
    print("\nEn Çok Karıştırılan Sınıflar:")
    for true_cls, pred_cls, count, pct in top_conf:
        print(f"Gerçek: {true_cls}, Tahmin: {pred_cls} - {count} örnek ({pct:.2%})")

# Karışıklık matrisi analizi yap
plot_confusion_matrix(y_test, y_pred_test, sorted(chunked_df['Etiket'].unique()))

# TF-IDF vektörleyicisinden en önemli kelimeler
def get_top_features_per_class(vectorizer, clf, class_labels, top_n=10):
    feature_names = vectorizer.get_feature_names_out()
    
    if hasattr(clf, 'coef_'):
        coefs = clf.coef_
    else:
        raise ValueError("Sınıflandırıcının coef_ özelliği yok")
    
    top_features = {}
    
    for i, class_name in enumerate(class_labels):
        # Bu sınıf için katsayıları al
        class_coefs = coefs[i]
        # En yüksek katsayılara sahip özelliklerin indislerini bul
        top_indices = np.argsort(class_coefs)[-top_n:]
        # Bu indislere karşılık gelen özellik isimlerini ve katsayılarını al
        top_features[class_name] = [(feature_names[idx], class_coefs[idx]) for idx in top_indices]
    
    return top_features

# Her sınıf için en ayırt edici kelimeleri alma
print("\nEn Ayırt Edici Kelimeler (Her Film Türü İçin):")
class_labels = best_model.named_steps['clf'].classes_
vectorizer = best_model.named_steps['tfidf']
classifier = best_model.named_steps['clf']

try:
    top_features = get_top_features_per_class(vectorizer, classifier, class_labels)
    
    for class_name, features in top_features.items():
        print(f"\n{class_name.upper()} türü için en ayırt edici kelimeler:")
        for feature, coef in sorted(features, key=lambda x: x[1], reverse=True):
            print(f"  - {feature}: {coef:.4f}")
except Exception as e:
    print(f"Özellik analizi sırasında hata: {e}")

# Final model performans özeti
print("\n" + "="*50)
print("FINAL MODEL PERFORMANS ÖZETİ")
print("="*50)
print(f"Eğitim Doğruluğu: {train_accuracy:.4f}")
print(f"Test Doğruluğu: {test_accuracy:.4f}")
print(f"Eğitim-Test Farkı: {train_test_diff:.4f}")
print(f"Ağırlıklı F1-Score: {test_report['weighted avg']['f1-score']:.4f}")
print("="*50)



Eğitim Verisi Performansı:
Eğitim Doğruluğu (Train Accuracy): 0.9584

Eğitim Verisi Sınıflandırma Raporu:
              precision    recall  f1-score   support

     aksiyon       0.98      0.97      0.98       263
   animasyon       0.98      0.98      0.98       323
 bilim kurgu       0.96      0.99      0.97       282
   distoptik       0.94      0.97      0.95       226
        dram       0.99      0.94      0.97       309
   fantastik       0.98      0.96      0.97       340
     gerilim       0.97      0.94      0.95       323
      komedi       0.96      0.95      0.95       422
       korku       0.90      0.98      0.94       243
       müzik       0.90      0.98      0.94       336
    polisiye       0.97      0.96      0.97       250
    romantik       0.96      0.91      0.93       362
       savaş       0.90      1.00      0.95       170
        spor       0.99      0.96      0.97       378
         suç       0.94      0.94      0.94       413
       tarih       0.99     

In [15]:
import joblib
from pathlib import Path

# Model kaydetme
# Modeli kaydet
model_path = Path(r"C:\Users\Melek\yapayZeka\FilmTemaAnaliziProje\Models\tam_veri_seti_model4.pkl")
joblib.dump(best_model, model_path)
print(f"Model kaydedildi: {model_path}")

Model kaydedildi: C:\Users\Melek\yapayZeka\FilmTemaAnaliziProje\Models\tam_veri_seti_model4.pkl
